<a href="https://colab.research.google.com/github/mr1er0y/Topic-Modelling-Neuro/blob/main/model_BERT_adapter_EWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combining Pre-Trained Adapters using AdapterFusion


Задача: По анотации статьи и определить темы, к которым она относится. 

Эта задача относится к Multi-label classification.

Для задачи многоклассовой классификации с несколькими метками (multi-label classification) требуется модифицировать данные и метки, чтобы алгоритм мог предсказывать несколько меток для каждого объекта.



In [ ]:
!pip install -U adapter-transformers  > /dev/null
!pip install datasets  > /dev/null

## Dataset Preprocessing

В первую очередь нужно скачать данные из Kaggle(у меня они хранятся на Google Drive) и произвести преобразования  
1. Токенизировать данные
2. Бинанизировать вектор тем с помощью MultiLabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

path = "drive/MyDrive/Data_arXiv/filtered_arxiv_db.csv"
SEED = 77

df = pd.read_csv(path)
df[['created_date', 'update_date']] = df[['created_date', 'update_date']].apply(pd.to_datetime)
df = df.drop(['versions', 'description', 'new_category', 'sub_category'], axis=1)
df.columns = ['id', 'title', 'authors', 'category', 'published_date', 'updated_date', 'abstract']
df["category"] = df["category"].apply(eval)
df.head()

<ipython-input-3-716c63935ed4>:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,id,title,authors,category,published_date,updated_date,abstract
0,704.0033,Convergence of the discrete dipole approximati...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-31,We performed a rigorous theoretical converge...
1,704.0038,The discrete dipole approximation: an overview...,"Maxim A. Yurkin, Alfons G. Hoekstra","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-30,We present a review of the discrete dipole a...
2,704.0275,Mapping radii of metric spaces,George M. Bergman (U.C.Berkeley),[math.MG],2008-03-28,2021-10-15,It is known that every closed curve of lengt...
3,704.0479,The affine part of the Picard scheme,T.Geisser,"[math.AG, math.KT]",2021-01-28,2021-01-29,We describe the maximal torus and maximal un...
4,704.0752,Actions for the Bosonic String with the Curved...,Davoud Kamani,[hep-th],2008-04-18,2020-08-21,At first we introduce an action for the stri...


In [ ]:
def preprocessing(data):
    data = data[data['abstract'].str.contains('paper has been withdrawn') == False]
    data['abstract'] = [a.strip() for a in data['abstract']]
    data['abstract'] = data['abstract'].str.replace('\n', ' ', regex = False).str.replace('\t', ' ', regex = False).str.replace(r'\s\s+', ' ', regex = True)
    data['abstract'] = data['abstract'].str.replace('([.,!?()])', r' \1 ')
    return data

df = preprocessing(df)
df.head()

<ipython-input-4-60440652729b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data['abstract'] = data['abstract'].str.replace('([.,!?()])', r' \1 ')


,id,title,authors,category,published_date,updated_date,abstract
0,704.0033,Convergence of the discrete dipole approximati...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-31,We performed a rigorous theoretical convergenc...
1,704.0038,The discrete dipole approximation: an overview...,"Maxim A. Yurkin, Alfons G. Hoekstra","[physics.optics, physics.comp-ph]",2022-03-29,2022-03-30,We present a review of the discrete dipole app...
2,704.0275,Mapping radii of metric spaces,George M. Bergman (U.C.Berkeley),[math.MG],2008-03-28,2021-10-15,It is known that every closed curve of length ...
3,704.0479,The affine part of the Picard scheme,T.Geisser,"[math.AG, math.KT]",2021-01-28,2021-01-29,We describe the maximal torus and maximal unip...
4,704.0752,Actions for the Bosonic String with the Curved...,Davoud Kamani,[hep-th],2008-04-18,2020-08-21,At first we introduce an action for the string...


In [ ]:
df['unifed_text'] = df['title'] + '[SEP]' + df['abstract'] + '[SEP]'

In [ ]:
# Extract the categories column as a list of lists
categories = []
for el in df["category"]:
    categories.extend(el)
categories = np.unique(categories)
num_categories = len(categories)

# Initialize the MultiLabelBinarizer and fit_transform the categories
mlb = MultiLabelBinarizer()
df['labels'] =  mlb.fit_transform(df["category"].values).tolist()

In [ ]:
len(categories)

170

In [ ]:
categories

array(['adap-org', 'alg-geom', 'astro-ph', 'astro-ph.CO', 'astro-ph.EP',
       'astro-ph.GA', 'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR',
       'chao-dyn', 'cmp-lg', 'comp-gas', 'cond-mat', 'cond-mat.dis-nn',
       'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cond-mat.other',
       'cond-mat.quant-gas', 'cond-mat.soft', 'cond-mat.stat-mech',
       'cond-mat.str-el', 'cond-mat.supr-con', 'cs.AI', 'cs.AR', 'cs.CC',
       'cs.CE', 'cs.CG', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY', 'cs.DB',
       'cs.DC', 'cs.DL', 'cs.DM', 'cs.DS', 'cs.ET', 'cs.FL', 'cs.GL',
       'cs.GR', 'cs.GT', 'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO',
       'cs.MA', 'cs.MM', 'cs.MS', 'cs.NA', 'cs.NE', 'cs.NI', 'cs.OH',
       'cs.OS', 'cs.PF', 'cs.PL', 'cs.RO', 'cs.SC', 'cs.SD', 'cs.SE',
       'cs.SI', 'cs.SY', 'dg-ga', 'econ.EM', 'econ.GN', 'econ.TH',
       'eess.AS', 'eess.IV', 'eess.SP', 'eess.SY', 'funct-an', 'gr-qc',
       'hep-ex', 'hep-lat', 'hep-ph', 'hep-th', 'math-ph', 'math.AC',
       'math.AG', 'm

In [ ]:
t = df["abstract"].apply(len)
print("Max:" + str(max(t)))
print("Mean:" + str(np.mean(t)))

Max:5216
Mean:1098.1211080387932


In [ ]:
df[["abstract", "labels"]].sample(2)

,abstract,labels
560010,The high computational complexity and high ene...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
221178,Weighted First-Order Model Counting ( WFOMC )...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
from transformers import BertTokenizer, DistilBertTokenizerFast
from datasets import Dataset
import torch

dataset = Dataset.from_pandas(df[["unifed_text", "labels"]].sample(300000, random_state=SEED))
dataset.set_format(type="torch", columns=["unifed_text", "labels"])

# https://stackoverflow.com/questions/66096703/running-huggingface-bert-tokenizer-on-gpu
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', lowercase=True)
# tokenizer.save_pretrained('/std-bert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', lowercase=True)



def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  # токенизируем текстовые данные
  return tokenizer(
      batch["unifed_text"],
      max_length=350,
      truncation=True,
      padding="max_length"
  )

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [ ]:
# dataset = dataset.remove_columns(['title', 'abstract'])
dataset

Dataset({
    features: ['unifed_text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 300000
})

In [ ]:
# dataset.to_json("drive/MyDrive/Data_arXiv/token_dataset.json")

In [ ]:
# dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

dataset = dataset.train_test_split( test_size=0.2, shuffle=True, seed=SEED)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


In [ ]:
del dataset

`labels` - это закодированные в бинарном виде темы статей. Он состоит из 0 и 1, где 1 на i позиции обозначает, что тема i соотвествует тексту, а 0 - нет.

`input_ids` - это последовательность чисел, которые представляют отдельные токены в тексте, каждый токен преобразуется в уникальное число, которое соответствует его позиции в словаре модели BERT. Это основной входной тензор, который передается в модель.

`token_type_ids` - это последовательность чисел, которая указывает, к какому из двух предложений относится каждый токен в input_ids (0 для первого предложения и 1 для второго). Это необходимо в многих задачах, например, в задачах сравнения двух предложений, где модель должна понимать, какой токен относится к какому предложению.

`attention_mask`  это последовательность чисел, которая указывает, какие токены должны быть проигнорированы моделью в процессе обработки текста. Она состоит из 0 и 1, где 1 обозначает, что токен должен быть учтен в модели, а 0 - игнорирован.

## Training
Мы используем предварительно обученную лучшую модель из `Hugging Face` и создаем экземпляр нашей модели с помощью `BertModelWithHeads`.

In [ ]:
from transformers import BertConfig, BertModelWithHeads


config = BertConfig.from_pretrained(
    "bert-base-uncased",
)
model = BertModelWithHeads.from_pretrained(
    "bert-base-uncased",
    config=config,

)

# model.save_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:269: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/bert/adapter_model.py:247: FutureWarning: This class has been renamed to `BertAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModelWithHeads: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModelWithHea

Теперь у нас есть все настроенное для загрузки нашей программы Adapter Fusion setup. Сначала мы загружаем из концентратора три адаптера, предварительно обученных различным задачам: `MultiNLI`, `QQP` и `QNLI`. Поскольку нам не нужны их заголовки предсказаний, мы передаем `with_head=False` методу загрузки. Далее мы добавляем новый слой fusion, который объединяет все адаптеры, которые мы только что загрузили. Наконец, мы добавляем новую классификационную рубрику для нашей целевой задачи сверху.

In [ ]:
from transformers import BertModelWithHeads


model.load_adapter("AdapterHub/bert-base-uncased-pf-ud_deprel", source="hf", load_as="ewt", with_head=False)

# Add a classification head for our target task
model.add_classification_head("ewt", num_labels=len(categories), multilabel=True)

model.train_adapter("ewt")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

training_args = TrainingArguments(
    learning_rate=1e-3,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result


trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 240000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7500
  Number of trainable parameters = 1615850


Step,Training Loss
200,0.064500
400,0.049200
600,0.033300
800,0.028800
1000,0.027300
1200,0.026100
1400,0.025700
1600,0.024800
1800,0.025000
2000,0.024000


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/ewt/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/ewt/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/ewt/head_config.json
Module weights saved in ./training_output/checkpoint-500/ewt/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/ewt/head_config.json
Module weights saved in ./training_output/checkpoint-500/ewt/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/ewt/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/ewt/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-1000/ewt/head_config.json
Module weights saved in ./training_output/checkpoint-1000/ewt/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-1000/ewt/head_conf

TrainOutput(global_step=7500, training_loss=0.02453182702064514, metrics={'train_runtime': 3234.5717, 'train_samples_per_second': 74.198, 'train_steps_per_second': 2.319, 'total_flos': 4.3980270768e+16, 'train_loss': 0.02453182702064514, 'epoch': 1.0})

In [ ]:
print(trainer.evaluate(), flush=True)

***** Running Evaluation *****
  Num examples = 60000
  Batch size = 32


{'eval_loss': 0.019196685403585434, 'eval_f1': 0.6357614502513567, 'eval_roc_auc': 0.7689648630852345, 'eval_accuracy': 0.35746666666666665, 'eval_runtime': 390.7155, 'eval_samples_per_second': 153.564, 'eval_steps_per_second': 4.799, 'epoch': 1.0}
